In [1]:
import numpy as np

import torch
import time
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.data.functional import to_map_style_dataset
from torchtext.vocab import build_vocab_from_iterator

In [2]:
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)


def collate_batch(batch, device):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.shape[0])
    label_list = torch.tensor(label_list, dtype=torch.int64)
    # Call cumsum() to get absolute offset (instead of relative offset)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


def train(dataloader, model, optimizer, criterion):
    model.train()
    total_acc, total_count = 0.0, 0.0
    log_interval = 500
    start_time = time.time()
    
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        pred_label = model(text, offsets)
        loss = criterion(pred_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_acc += (pred_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(f"| epoch {epoch} | {idx} / {len(dataloader)} batches | train acc {np.round(total_acc / total_count, 2)}")
        total_acc, total_count = 0.0, 0.0
        start_time = time.time()
        

def evaluate(dataloader, model, criterion):
    model.eval()
    total_acc, total_count = 0.0, 0.0
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            pred_label = model(text, offsets)
            loss = criterion(pred_label, label)
            total_acc += (pred_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
        return total_acc / total_count

In [3]:
class TextClassifier(nn.Module):
    
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_class)
        self.init_weights()
        
    def init_weights(self):
        # Use Leaky ReLU since behavior is similar to ELU 
        nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity="leaky_relu")
        nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)
        
    def forward(self, text, offsets):
        embedded = self.embedding.forward(text, offsets)
        x = self.fc1(embedded)
        x = F.elu(x)
        x = self.fc2(x)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_iter = AG_NEWS(split="train")
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [6]:
num_class = len(set([label for label, text in train_iter]))
vocab_size = len(vocab)
emsize = 64
hdsize = emsize
model = TextClassifier(vocab_size, emsize, hdsize, num_class).to(device)

In [7]:
model

TextClassifier(
  (embedding): EmbeddingBag(95811, 64, mode='mean')
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=4, bias=True)
)

In [8]:
for i, (_, txt) in enumerate(train_iter):
    print(f">>>\t{txt}")
    print(f">>>\t{tokenizer(txt)}")
    print("")
    if i == 1:
        break

>>>	Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
>>>	['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.']

>>>	Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
>>>	['carlyle', 'looks', 'toward', 'commercial', 'aerospace', '(', 'reuters', ')', 'reuters', '-', 'private', 'investment', 'firm', 'carlyle', 'group', ',', '\\which', 'has', 'a', 'reputation', 'for', 'making', 'well-timed', 'and', 'occasionally\\controversial', 'plays', 'in', 'the', 'defense', 'industry', ',', 'has', 'quietly', 'p

In [9]:
print(
    vocab(["my", "dog", "'", "s", "name", "is", "fido"])
)

print(
    text_pipeline("My dog's name is Fido")
)

print(
    label_pipeline("9")
)

[1300, 5383, 16, 9, 951, 21, 45387]
[1300, 5383, 16, 9, 951, 21, 45387]
8


In [10]:
epochs = 10
lr = 5.0
batch_size = 64

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1.0, gamma=0.1)

In [12]:
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=batch_size, shuffle=True, collate_fn=lambda x: collate_batch(x, device))
valid_dataloader = DataLoader(split_valid_, batch_size=batch_size, shuffle=True, collate_fn=lambda x: collate_batch(x, device))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: collate_batch(x, device))

In [13]:
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion)
    accu_val = evaluate(valid_dataloader, model, criterion)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(f"| end of epoch {epoch} | time: {time.time() - epoch_start_time} | valid acc {np.round(accu_val, 2)}")

| epoch 1 | 500 / 1782 batches | train acc 0.72
| epoch 1 | 1000 / 1782 batches | train acc 0.73
| epoch 1 | 1500 / 1782 batches | train acc 0.77
-----------------------------------------------------------
| end of epoch 1 | time: 9.99433445930481 | valid acc 0.77
| epoch 2 | 500 / 1782 batches | train acc 0.8
| epoch 2 | 1000 / 1782 batches | train acc 0.81
| epoch 2 | 1500 / 1782 batches | train acc 0.92
-----------------------------------------------------------
| end of epoch 2 | time: 9.720261096954346 | valid acc 0.83
| epoch 3 | 500 / 1782 batches | train acc 0.94
| epoch 3 | 1000 / 1782 batches | train acc 0.86
| epoch 3 | 1500 / 1782 batches | train acc 0.92
-----------------------------------------------------------
| end of epoch 3 | time: 9.638692140579224 | valid acc 0.86
| epoch 4 | 500 / 1782 batches | train acc 0.81
| epoch 4 | 1000 / 1782 batches | train acc 0.83
| epoch 4 | 1500 / 1782 batches | train acc 0.92
----------------------------------------------------------